In [1]:
import time
import numpy as np
import syncmetrics as syncm
from numpy_ringbuffer import RingBuffer

from pythonosc import dispatcher
from pythonosc import osc_server
from pythonosc import udp_client
from pythonosc import osc_message_builder
from pythonosc import osc_bundle_builder

In [2]:
class Session:
    # to do processing in windoes 
    fs=1000
    buffer_W=150
    win_size=15*fs
    W_A = np.zeros(win_size)
    W_B = np.zeros(win_size)
    no_buffer=int(win_size/buffer_W)
    delta_t=0.1*fs
    j = 0
    
#OSC input config   
ip = 'localhost'
port = 8001
server = osc_server.ThreadingOSCUDPServer(
  (ip, port), dispatcher)
    
# OSC output config
local_ip = '192.168.3.105'
local_port = 12000
output_address = '/results'
client = udp_client.SimpleUDPClient(local_ip, local_port)

In [3]:
#    Send to Processing/MAX/Etc...
def send_osc(dev_id, *values):
    bundle = osc_bundle_builder.OscBundleBuilder(osc_bundle_builder.IMMEDIATELY)
    msg = osc_message_builder.OscMessageBuilder(address=output_address)
    # Test bundle outputs: 1, 2, 3, ...
    for val in values:
        print (val)
        msg.add_arg(val)
        bundle.add_content(msg.build())
        bundle = bundle.build()
    client.send(bundle)
    
def output_individual(data, labels):
    data = json.loads(data)
    #printJSON(data)
    bundle = osc_bundle_builder.OscBundleBuilder(
        osc_bundle_builder.IMMEDIATELY)

    for label in labels:
        output_address += self.device_number + "/bitalino/" + label
        msg = osc_message_builder.OscMessageBuilder(
            address=output_address)
        msg.add_arg(arg_to_add)
        msg.build()
        client.send(msg)

In [4]:
global sigA
global sigB
global W_A
global W_B

window = Session()

def bitalino_input_handler(unused_addr, *values):
    dataframe = []
    # Store and normalise data
    # Store and normalise data
    data = values
#     print (data)
#                        bit1.RESP1 = (bit1.val("RESP1")-512)/1024
    sigA = data[-2]
    sigB = data[-1]
    if window.j < window.no_buffer :
        i = window.j % window.no_buffer
        window.W_A[i*window.buffer_W:(i+1)*window.buffer_W]=sigA
        window.W_B[i*window.buffer_W:(i+1)*window.buffer_W]=sigB
        window.j = window.j+1

    else :
        window.W_A = np.roll(window.W_A,-window.buffer_W)
        window.W_A[-window.buffer_W:] = sigA

        window.W_B = np.roll(window.W_B,-window.buffer_W)
        window.W_B[-window.buffer_W:] = sigB
        results = []
        result_1=syncm.lin_reg_r_metric(window.W_A,window.W_B)
        result_2=syncm.cos_similarity(window.W_A,window.W_B)

        #add results here
        results = [result_1, result_2.flat[0]]
#         resuts {
#             'reg': syncm.lin_reg_r_metric(window.W_A,window.W_B,
#             'cos': result_2
#         }
        print(results)
        #forward to OSC bundle
        send_osc(0, results)

#     for line in dataframe.values:
#         self.txtFile.addData('\n')
#         self.txtFile.addData(",".join([str(x) for x in line]))

In [5]:
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/*/bitalino", bitalino_input_handler)

In [ ]:
print("Serving on {}".format(server.server_address))
server.serve_forever()

Serving on ('127.0.0.1', 8001)
[0.15555049785600725, 0.9999983514346309]
[0.15555049785600725, 0.9999983514346309]
[0.15733671278590233, 0.9999983082065939]
[0.15733671278590233, 0.9999983082065939]
[0.1593244411827338, 0.9999982698856604]
[0.1593244411827338, 0.9999982698856604]
[0.16150789101030408, 0.9999982364738921]
[0.16150789101030408, 0.9999982364738921]
[0.16388422727548146, 0.9999982079733566]
[0.16388422727548146, 0.9999982079733566]
[0.0, 0.999998142760381]
[0.0, 0.999998142760381]
[0.0, 0.9999981228111867]
[0.0, 0.9999981228111867]
[0.0, 0.9999980649261535]
[0.0, 0.9999980649261535]
[0.0, 0.9999980535770411]
[0.0, 0.9999980535770411]
[0.0, 0.9999980471488086]
[0.0, 0.9999980471488086]
[0.0, 0.9999980456435273]
[0.0, 0.9999980456435273]
[0.0, 0.9999980490632686]
[0.0, 0.9999980490632686]
[0.0, 0.9999980086806285]
[0.0, 0.9999980086806285]
[0.0, 0.9999980207176717]
[0.0, 0.9999980207176717]
[0.0, 0.9999980376854402]
[0.0, 0.9999980376854402]
[0.0, 0.9999980074497361]
[0.0, 0

[0.702208183006853, 0.9999982658875185]
[0.702208183006853, 0.9999982658875185]
[0.6999365242268759, 0.9999982770768581]
[0.6999365242268759, 0.9999982770768581]
[0.6973279974221717, 0.9999982910436828]
[0.6973279974221717, 0.9999982910436828]
[0.6943650748294136, 0.9999983077894767]
[0.6943650748294136, 0.9999983077894767]
[0.6910272623814477, 0.9999983273157266]
[0.6910272623814477, 0.9999983273157266]
[0.6872907035374676, 0.9999983496239159]
[0.6872907035374676, 0.9999983496239159]
[0.68312769552246, 0.9999983747155291]
[0.68312769552246, 0.9999983747155291]
[0.6785060971688444, 0.9999984025920559]
[0.6785060971688444, 0.9999984025920559]
[0.6733886011577476, 0.9999984332549825]
[0.6733886011577476, 0.9999984332549825]
[0.667731834772177, 0.9999984667057988]
[0.667731834772177, 0.9999984667057988]
[0.6614852413169074, 0.9999985029459917]
[0.6614852413169074, 0.9999985029459917]
[0.6545896776522966, 0.9999985419770532]
[0.6545896776522966, 0.9999985419770532]
[0.6469756395693392, 0.9

[-0.08819719603295167, 0.9999984307388567]
[-0.08819719603295167, 0.9999984307388567]
[-0.09236217287825793, 0.9999984653339422]
[-0.09236217287825793, 0.9999984653339422]
[-0.09673690413940701, 0.9999985027953953]
[-0.09673690413940701, 0.9999985027953953]
[-0.10134955055890435, 0.9999985431241272]
[-0.10134955055890435, 0.9999985431241272]
[-0.10623320276723167, 0.999998586321065]
[-0.10623320276723167, 0.999998586321065]
[-0.11142711705324408, 0.9999986323871288]
[-0.11142711705324408, 0.9999986323871288]
[-0.11697834638775186, 0.9999986813232384]
[-0.11697834638775186, 0.9999986813232384]
[-0.12294392655508786, 0.9999987331303135]
[-0.12294392655508786, 0.9999987331303135]
[-0.12939385676077383, 0.9999987878092771]
[-0.12939385676077383, 0.9999987878092771]
[-0.1321453204217, 0.9999987940702062]
[-0.1321453204217, 0.9999987940702062]
[-0.13498962844602877, 0.9999988016036776]
[-0.13498962844602877, 0.9999988016036776]
[-0.1379334722460226, 0.999998810409958]
[-0.1379334722460226, 0

[-0.6294193789010628, 0.9999983607676366]
[-0.6294193789010628, 0.9999983607676366]
[-0.6155087814372388, 0.9999983656515633]
[-0.6155087814372388, 0.9999983656515633]
[-0.6083501120188276, 0.9999984026526241]
[-0.6083501120188276, 0.9999984026526241]
[-0.6006537340759428, 0.9999984409525228]
[-0.6006537340759428, 0.9999984409525228]
[-0.5923715332286773, 0.9999984805512621]
[-0.5923715332286773, 0.9999984805512621]
[-0.5765767848191621, 0.9999984941639866]
[-0.5765767848191621, 0.9999984941639866]
[-0.5606770793101042, 0.9999985106802929]
[-0.5606770793101042, 0.9999985106802929]
[-0.5445349874392269, 0.9999985300998814]
[-0.5445349874392269, 0.9999985300998814]
[-0.5330373754213998, 0.9999985768213253]
[-0.5330373754213998, 0.9999985768213253]
[-0.540983683784695, 0.9999985442834964]
[-0.540983683784695, 0.9999985442834964]
[-0.5227659422079896, 0.999998572431183]
[-0.5227659422079896, 0.999998572431183]
[-0.5285202725982147, 0.9999985489717959]
[-0.5285202725982147, 0.99999854897179

[-0.3611210013440812, 0.9999988083675024]
[-0.3611210013440812, 0.9999988083675024]
[-0.367028531006772, 0.9999987835552336]
[-0.367028531006772, 0.9999987835552336]
[-0.32860621332774786, 0.9999988662262794]
[-0.32860621332774786, 0.9999988662262794]
[-0.3352916705557035, 0.9999988452207756]
[-0.3352916705557035, 0.9999988452207756]
[-0.350641510979986, 0.9999988064389532]
[-0.350641510979986, 0.9999988064389532]
[-0.3311758133602751, 0.9999988299450501]
[-0.3311758133602751, 0.9999988299450501]
[-0.34299013356513297, 0.9999988002267464]
[-0.34299013356513297, 0.9999988002267464]
[-0.3525417989183212, 0.9999987756961349]
[-0.3525417989183212, 0.9999987756961349]
[-0.28017574407289647, 0.9999988303780473]
[-0.28017574407289647, 0.9999988303780473]
[-0.28716979207234583, 0.9999988135952751]
[-0.28716979207234583, 0.9999988135952751]
[-0.26341818190263167, 0.9999988719888779]
[-0.26341818190263167, 0.9999988719888779]
[-0.26506779756002774, 0.9999988669410727]
[-0.26506779756002774, 0.99

...

[0.0, 0.9999998961339583]
[0.0, 0.9999998961339583]
[0.0, 0.9999999125208092]
[0.0, 0.9999999125208092]
[0.0, 0.9999999125208092]
[0.0, 0.9999999125208092]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772638]
[0.0, 0.9999999292772638]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772638]
[0.0, 0.9999999292772638]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999999292772639]
[0.0, 0.9999

[-0.2072490727586874, 0.9999977113787999]
[-0.2072490727586874, 0.9999977113787999]
[-0.201259304286126, 0.9999977365466097]
[-0.201259304286126, 0.9999977365466097]
[-0.19354843268373315, 0.9999977629537756]
[-0.19354843268373315, 0.9999977629537756]
[-0.18558590561995747, 0.999997790817921]
[-0.18558590561995747, 0.999997790817921]
[-0.18354091113605242, 0.9999978233392899]
[-0.18354091113605242, 0.9999978233392899]
[-0.1814998119018472, 0.9999978562218782]
[-0.1814998119018472, 0.9999978562218782]
[-0.17946264951384364, 0.9999978894657233]
[-0.17946264951384364, 0.9999978894657233]
[-0.17050245715126, 0.9999979209613398]
[-0.17050245715126, 0.9999979209613398]
[-0.16121302762782483, 0.9999979539141024]
[-0.16121302762782483, 0.9999979539141024]
[-0.1586539614146976, 0.999997988969184]
[-0.1586539614146976, 0.999997988969184]
[-0.1586539614146975, 0.9999979889691839]
[-0.1586539614146975, 0.9999979889691839]
[-0.15608127311788678, 0.9999980243857158]
[-0.15608127311788678, 0.99999802

[-0.19577488771641757, 0.9999977368205109]
[-0.19577488771641757, 0.9999977368205109]
[-0.20165518385118364, 0.9999977338749706]
[-0.20165518385118364, 0.9999977338749706]
[-0.19577488771641757, 0.9999977338749706]
[-0.19577488771641757, 0.9999977338749706]
[-0.19471559175704636, 0.9999977650863299]
[-0.19471559175704636, 0.9999977650863299]
[-0.20051165202344523, 0.9999977632375382]
[-0.20051165202344523, 0.9999977632375382]
[-0.20051165202344529, 0.9999977632375381]
[-0.20051165202344529, 0.9999977632375381]
[-0.2063686187725281, 0.9999977617559395]
[-0.2063686187725281, 0.9999977617559395]
[-0.19294749245710813, 0.9999977966299485]
[-0.19294749245710813, 0.9999977966299485]
[-0.2009055432359175, 0.9999977955773506]
[-0.2009055432359175, 0.9999977955773506]
[-0.2090946466916998, 0.9999977948834735]
[-0.2090946466916998, 0.9999977948834735]
[-0.21451273314742542, 0.9999977948570244]
[-0.21451273314742542, 0.9999977948570244]
[-0.2228051230650705, 0.9999977948848328]
[-0.22280512306507

[0.15482063376916808, 0.9999998006280371][0.15197361672544302, 0.9999997813508242]
[0.15197361672544302, 0.9999997813508242]
[0.14992025991090982, 0.9999997624421783]
[0.14992025991090982, 0.9999997624421783]

[0.15482063376916808, 0.9999998006280371]
[0.1499202599109086, 0.9999997624421784]
[0.1499202599109086, 0.9999997624421784]
[0.14992025991091024, 0.9999997624421783][0.12457698694251562, 0.9999997443071924]
[0.12457698694251562, 0.9999997443071924]
[0.14992025991091024, 0.9999997624421783]

[0.1245769869425156, 0.9999997443071923]
[0.1245769869425156, 0.9999997443071923]
[0.10667614941253326, 0.9999997265296521]
[0.10667614941253326, 0.9999997265296521]
[-0.02109452895664322, 0.9999996553697308]
[-0.02109452895664322, 0.9999996553697308]
[-0.013523807642946565, 0.9999996382875598]
[-0.013523807642946565, 0.9999996382875598]
[-0.01352380764294774, 0.9999996382875599]
[-0.01352380764294774, 0.9999996382875599]
[-0.014685706089122876, 0.9999996219502677]
[-0.014685706089122876, 0.99

[0.4070761712189886, 0.9999996402478906]
[0.4070761712189886, 0.9999996402478906]
[0.40707617121898876, 0.9999996402478906]
[0.40707617121898876, 0.9999996402478906]
[0.3830437673586622, 0.9999996402763196]
[0.3830437673586622, 0.9999996402763196]
[0.35615184440317815, 0.9999996403047797]
[0.35615184440317815, 0.9999996403047797]
[0.3259000380632609, 0.9999996403332689]
[0.3259000380632609, 0.9999996403332689]
[0.2916619944306952, 0.9999996403617836]
[0.2916619944306952, 0.9999996403617836]
[0.2526431814162523, 0.9999996403903306]
[0.2526431814162523, 0.9999996403903306]
[0.19418390934515406, 0.9999996240483896]
[0.19418390934515406, 0.9999996240483896]
[0.2078205205718731, 0.9999996404189081]
[0.2078205205718731, 0.9999996404189081]
[0.20782052057187161, 0.9999996404189083]
[0.20782052057187161, 0.9999996404189083]
[0.15585399336633612, 0.9999996404475173]
[0.15585399336633612, 0.9999996404475173]
[0.16922612767541384, 0.9999996571717976]
[0.16922612767541384, 0.9999996571717976]
[0.1